In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2OXGBoostEstimator
import random # H2O 초기화 시 포트 번호 충돌 방지를 위해 사용

import mlflow
import mlflow.h2o# H2O 모델을 MLflow에 로깅하기 위한 H2O Flavor

from mlflow.models.signature import infer_signature 
from pyspark.sql import functions as F

In [0]:
# 2. CSV 데이터 불러오기
# 데이터 경로를 실제 환경에 맞게 수정해주세요.
csv_path = '/Volumes/project/default/project/UHI_data_test.csv/'
df = spark.read.option("header", "true").csv(csv_path)

print("--- 원본 Spark DataFrame 스키마 ---")
df.printSchema()
df.show(5, truncate=False)

In [0]:
# 3. 데이터 타입 변환 (df_processed 생성)
# H2O AutoML이 datetime 타입의 timestamp를 인식하도록 cast 합니다.
df_processed = df.withColumn("green_rate", col("green_rate").cast("double")) \
                  .withColumn("Building_Density", col("Building_Density").cast("double")) \
                  .withColumn("car_registration_count", col("car_registration_count").cast("long")) \
                  .withColumn("population_density", col("population_density").cast("double")) \
                  .withColumn("avg_km_per_road_km", col("avg_km_per_road_km").cast("double")) \
                  .withColumn("UHII", col("UHII").cast("double")) \
                  .withColumn("timestamp", col("timestamp").cast("timestamp")) \
                  .withColumn("suburban_temp_current", col("suburban_temp_current").cast("double"))

print("\n--- 타입 변환 후 Spark DataFrame 스키마 ---")
df_processed.printSchema()
df_processed.show(5, truncate=False)

In [0]:
# 4. Spark DataFrame을 Pandas DataFrame으로 변환
# H2O는 Pandas DataFrame 또는 H2OFrame을 입력으로 받습니다.
# 대규모 데이터셋의 경우 메모리 부족에 주의해야 합니다.
try:
    pandas_df = df_processed.toPandas()
    print(f"\nSpark DataFrame을 Pandas DataFrame으로 변환 완료. 크기: {pandas_df.shape}")
except Exception as e:
    print(f"Pandas DataFrame 변환 중 오류 발생: {e}")
    print("데이터셋이 너무 커서 메모리 부족이 발생했을 수 있습니다. 샘플링을 고려하세요.")
    # 예: pandas_df = df_processed.sample(fraction=0.1, seed=42).toPandas()

In [0]:
# 5. H2O 초기화
# Databricks에서 H2O를 초기화할 때 포트 충돌이 발생할 수 있으므로, 랜덤 포트를 사용하거나 특정 포트를 지정하는 것이 좋습니다.
# cluster_mode=True 설정은 H2O를 Spark 클러스터에서 실행합니다. (권장)
# nthreads=-1은 사용 가능한 모든 코어를 사용하겠다는 의미입니다.
try:
    h2o.init(
        # ip="localhost", # 로컬 IP 주소 지정 (선택 사항)
        port=random.randint(54321, 65535), # 랜덤 포트 사용
        nthreads=-1, # 사용 가능한 모든 스레드 사용
        min_mem_size="4G", # H2O에 할당할 최소 메모리 크기 (클러스터 환경에 맞게 조정)
        # spark_session=spark, # SparkSession 지정 (Spark 클러스터에서 H2O 실행 시)
        # strict_version_check=False # H2O 버전 체크 비활성화 (환경 문제시)
    )
    print("\nH2O Flow UI: %s" % h2o.cluster().get_web_ui())
except Exception as e:
    print(f"H2O 초기화 중 오류 발생: {e}")
    print("H2O 클러스터가 이미 실행 중이거나 포트 충돌이 발생했을 수 있습니다.")
    print("h2o.shutdown() 후 다시 시도하거나, 다른 포트 번호를 사용해보세요.")

In [0]:
# 6. Pandas DataFrame을 H2OFrame으로 변환
h2o_frame = h2o.H2OFrame(pandas_df)

print("\nH2OFrame 정보:")
h2o_frame.summary()

In [0]:
# 7. 예측 대상 (y) 및 피처 (x) 정의
y = 'UHII' # 예측하려는 컬럼
x = [col for col in h2o_frame.columns if col != y]

# H2OFrame의 컬럼 타입을 확인하는 올바른 방법은 .types 딕셔너리를 사용하는 것입니다.
timestamp_col_type = h2o_frame.types.get('timestamp')

print(f"\n'{y}'는 예측 대상입니다.")
print(f"사용될 피처들 ({len(x)}개): {x}")
print(f"H2OFrame의 'timestamp' 컬럼 타입: {timestamp_col_type}") # 올바른 타입 출력

# 이제 이 'timestamp_col_type' 변수를 사용하여 조건을 확인합니다.
if timestamp_col_type != 'time':
    print("경고: 'timestamp' 컬럼이 'time' 타입으로 인식되지 않았습니다. 변환을 다시 확인하거나 데이터 형식을 점검하세요.")
    # 만약 여전히 'time'이 아니라면, 다음 줄의 주석을 해제하여 강제로 'time' 타입으로 변환 시도
    # h2o_frame['timestamp'] = h2o_frame['timestamp'].as_time()
    # print(f"강제 변환 후 'timestamp' 컬럼 타입: {h2o_frame.types.get('timestamp')}")

In [0]:
# 8. AutoML 모델 학습 설정 및 실행
# max_runtime_secs: 최대 실행 시간 (초). 시간 부족 시 모델 수가 적을 수 있습니다.
# max_models: 최대 훈련할 모델 수.
# seed: 재현성을 위한 시드 값.
# sort_metric: 리더보드를 정렬하고 최적 모델을 선택할 때 사용할 평가 지표. (회귀: rmse, mae, rmsle, mean_residual_deviance)
# exclude_algos: 특정 알고리즘 제외 (선택 사항).
# include_algos: 특정 알고리즘만 포함 (선택 사항).
# nfolds: 교차 검증 폴드 수.
# verbosity: 상세 출력 수준 (debug, info, warn, error).

aml = H2OAutoML(
    max_runtime_secs=1800,  # 예: 1시간 (3600초) 동안 실행
    max_models=40,         # 예: 최대 20개의 모델 훈련
    seed=42,                # 재현성을 위한 시드
    sort_metric="rmse",     # 회귀 문제이므로 RMSE 사용
    # exclude_algos=["DeepLearning", "GLM"], # 특정 알고리즘 제외 예시
    # nfolds=5, # 교차 검증을 5-fold로 설정 (선택 사항, 기본값은 자동으로 설정될 수 있음)
    # verbosity="info", # 상세 정보 출력
    # project_name="UHI_Prediction" # 프로젝트 이름 지정
)

print("\n--- H2O AutoML 학습 시작 ---")
aml.train(x=x, y=y, training_frame=h2o_frame)

print("\n--- H2O AutoML 학습 완료 ---")

In [0]:
# 9. 리더보드 확인 (최고 성능 모델 목록)
leaderboard = aml.leaderboard
print("\n--- AutoML Leaderboard ---")
leaderboard.head(rows=10) # 상위 10개 모델 출력

# 10. 최고 성능 모델 확인
best_model = aml.leader
print(f"\n최고 성능 모델: {best_model.model_id}")

# 모델 성능 평가 (학습 데이터 기준)
print(f"\n--- 최고 모델 ({best_model.model_id})의 학습 데이터 기준 성능 ---")
print(f"RMSE (Root Mean Squared Error): {best_model.rmse(train=True)}")
print(f"R2 (R-squared): {best_model.r2(train=True)}")
print(f"MAE (Mean Absolute Error): {best_model.mae(train=True)}")
print(f"RMSLE (Root Mean Squared Logarithmic Error): {best_model.rmsle(train=True)}") # 필요한 경우 사용
print(f"MRD (Mean Residual Deviance): {best_model.mean_residual_deviance(train=True)}")


# 만약 검증 데이터를 분리했다면:
# print(f"최고 모델 ({best_model.model_id})의 검증 데이터 RMSE: {best_model.rmse(valid=True)}")

In [0]:
# 피쳐 중요도
# (이전 코드에서 H2OAutoML 학습 완료 후 best_model 객체가 있다고 가정)
# best_model = aml.leader

# 피처 중요도 테이블 가져오기
varimp_df = best_model.varimp(use_pandas=True)

print("\n--- 최고 모델의 피처 중요도 (상위 10개) ---")
# Importance, Percentage, Cumulative Sum 컬럼이 포함됩니다.
print(varimp_df.head(10))

# 전체 피처 중요도를 보고 싶다면
# print(varimp_df)

# 중요도 순으로 정렬되어 있으므로, 처음 몇 개만 봐도 주요 피처를 알 수 있습니다.

In [0]:
h2o_df=h2o_frame

In [0]:
# 모델 저장
save_path = h2o.save_model(model=best_model, path="/dbfs/tmp/mymodel", force=True)
print(f"모델 저장 경로: {save_path}")

In [0]:
model_path="/dbfs/tmp/mymodel/XGBoost_1_AutoML_2_20250605_80504"

In [0]:
# 모델 로드
best_model = h2o.load_model(model_path)

## 모델 예측

In [0]:
# 모델 예측 테스트

area = 'Gangnam-gu'
val1 = 0.4
val2 = 1.3
val3 = 257346
val4 = 14258.6
val5 = 20753.1

new_data_spark_df_raw = spark.createDataFrame([
    (area, val1, val2, val3, val4, val5, '2024-06-09T12:00:00.000Z', 26.0),
    (area, val1, val2, val3, val4, val5, '2024-06-09T15:00:00.000Z', 20.9),
    (area, val1, val2, val3, val4, val5, '2024-06-09T18:00:00.000Z', 20.9),
    (area, val1, val2, val3, val4, val5, '2024-06-09T21:00:00.000Z', 20.9),

], schema=[
    'District', 'green_rate', 'Building_Density', 'car_registration_count',
    'population_density', 'avg_km_per_road_km', 'timestamp', 'suburban_temp_current'
])

# 2. 학습 데이터와 동일한 Spark DataFrame 전처리 적용
# 여기서 timestamp 컬럼을 Spark의 timestamp 타입으로 캐스팅합니다.
new_data_spark_df_processed = new_data_spark_df_raw.withColumn("green_rate", col("green_rate").cast("double")) \
                                                     .withColumn("Building_Density", col("Building_Density").cast("double")) \
                                                     .withColumn("car_registration_count", col("car_registration_count").cast("long")) \
                                                     .withColumn("population_density", col("population_density").cast("double")) \
                                                     .withColumn("avg_km_per_road_km", col("avg_km_per_road_km").cast("double")) \
                                                     .withColumn("timestamp", col("timestamp").cast("timestamp")) \
                                                     .withColumn("suburban_temp_current", col("suburban_temp_current").cast("double"))

print("\n--- 예측할 새로운 Spark DataFrame (전처리 후) ---")

# 3. Spark DataFrame을 Pandas DataFrame으로 변환
new_data_pandas_df = new_data_spark_df_processed.toPandas()

# Pandas에서 datetime 타입으로 이미 잘 변환되었는지 확인
print("\n--- 예측할 새로운 Pandas DataFrame ---")
display(new_data_pandas_df)
print("\nTimestamp 컬럼 타입 (Pandas):", new_data_pandas_df['timestamp'].dtype)


# 4. Pandas DataFrame을 H2OFrame으로 변환 (예측 데이터)
# 학습 시와 동일한 방식으로 H2O가 'timestamp' 컬럼을 'time' 타입으로 자동 추론하기를 기대합니다.
new_h2o_frame = h2o.H2OFrame(new_data_pandas_df)


# 5. H2OFrame의 'timestamp' 컬럼 타입 확인
# H2O가 'time' 타입으로 잘 인식했는지 최종 확인합니다.
h2o_frame_timestamp_type = new_h2o_frame.types.get('timestamp')
#print(f"\n변환 후 예측 데이터 'timestamp' 컬럼 H2O 타입: {h2o_frame_timestamp_type}")

if h2o_frame_timestamp_type != 'time':
    print("경고: 'timestamp' 컬럼이 'time' 타입으로 인식되지 않았습니다. 모델 예측에 문제가 발생할 수 있습니다.")

# 6. 최고 성능 모델로 예측 수행
predictions = best_model.predict(new_h2o_frame)

# 7. 예측 결과를 Pandas DataFrame으로 변환하여 원본 데이터와 함께 보기
predictions_df = predictions.as_data_frame()
result_df = pd.concat([new_data_pandas_df, predictions_df], axis=1)


print("\n--- 입력 데이터와 최종 UHII 예측 결과 ---")
display(result_df)

In [0]:
# 예측 - 25개구 차이 확인

seoul_districts = [
    'Gangnam-gu', 'Gangdong-gu', 'Gangbuk-gu', 'Gangseo-gu',
    'Gwanak-gu', 'Gwangjin-gu', 'Guro-gu', 'Geumcheon-gu',
    'Nowon-gu', 'Dobong-gu', 'Dongdaemun-gu', 'Dongjak-gu',
    'Mapo-gu', 'Seodaemun-gu', 'Seocho-gu', 'Seongdong-gu',
    'Seongbuk-gu', 'Songpa-gu', 'Yangcheon-gu', 'Yeongdeungpo-gu',
    'Yongsan-gu', 'Eunpyeong-gu', 'Jongno-gu', 'Jung-gu', 'Jungnang-gu'
]

# 모든 구에 적용할 공통 조건 값 (수동으로 설정)
# 이 값들은 예시이며, 특정 시나리오에 맞게 조정할 수 있습니다.
common_green_rate = 0.45          # 중간 정도의 녹지율
common_building_density = 1.3     # 중간 정도의 건물 밀도
common_car_registration_count = 100000 # 일반적인 차량 등록 대수
common_population_density = 18000.0 # 일반적인 인구 밀도
common_avg_km_per_road_km = 8000.0  # 일반적인 도로 km당 주행 거리
common_timestamp = '2025-08-01T16:00:00.000Z' # 여름 오후 4시
common_suburban_temp_current = 23.5 # 여름철 다소 더운 외곽 온도

# 25개 구 각각에 대해 동일한 수치형 조건과 시간/외곽 온도를 가진 데이터를 생성
data_rows = []
for district in seoul_districts:
    data_rows.append((
        district,
        common_green_rate,
        common_building_density,
        common_car_registration_count,
        common_population_density,
        common_avg_km_per_road_km,
        common_timestamp,
        common_suburban_temp_current
    ))

# new_data_spark_df_raw 변수에 위에서 생성한 data_rows를 할당합니다.
new_data_spark_df_raw = spark.createDataFrame(data_rows, schema=[
    'District', 'green_rate', 'Building_Density', 'car_registration_count',
    'population_density', 'avg_km_per_road_km', 'timestamp', 'suburban_temp_current'
])

# 2. 학습 데이터와 동일한 Spark DataFrame 전처리 적용
# 이 부분은 변경되지 않습니다.
new_data_spark_df_processed = new_data_spark_df_raw.withColumn("green_rate", col("green_rate").cast("double")) \
                                                     .withColumn("Building_Density", col("Building_Density").cast("double")) \
                                                     .withColumn("car_registration_count", col("car_registration_count").cast("long")) \
                                                     .withColumn("population_density", col("population_density").cast("double")) \
                                                     .withColumn("avg_km_per_road_km", col("avg_km_per_road_km").cast("double")) \
                                                     .withColumn("timestamp", col("timestamp").cast("timestamp")) \
                                                     .withColumn("suburban_temp_current", col("suburban_temp_current").cast("double"))

# 3. Spark DataFrame을 Pandas DataFrame으로 변환
# 이 부분은 변경되지 않습니다.
new_data_pandas_df = new_data_spark_df_processed.toPandas()

print("\n--- 예측할 새로운 Pandas DataFrame ---")
display(new_data_pandas_df)
print("\nTimestamp 컬럼 타입 (Pandas):", new_data_pandas_df['timestamp'].dtype)

# 4. Pandas DataFrame을 H2OFrame으로 변환 (예측 데이터)
# best_model 객체는 이전에 학습이 완료되어 정의되어 있어야 합니다.
# 이 부분은 변경되지 않습니다.
new_h2o_frame = h2o.H2OFrame(new_data_pandas_df)

# 5. H2OFrame의 'timestamp' 컬럼 타입 확인
# 이 부분은 변경되지 않습니다.
h2o_frame_timestamp_type = new_h2o_frame.types.get('timestamp')
print(f"\n변환 후 예측 데이터 'timestamp' 컬럼 H2O 타입: {h2o_frame_timestamp_type}")

if h2o_frame_timestamp_type != 'time':
    print("경고: 'timestamp' 컬럼이 'time' 타입으로 인식되지 않았습니다. 모델 예측에 문제가 발생할 수 있습니다.")

# 6. 최고 성능 모델로 예측 수행
predictions = best_model.predict(new_h2o_frame)

# 7. 예측 결과를 Pandas DataFrame으로 변환하여 원본 데이터와 함께 보기
predictions_df = predictions.as_data_frame()
result_df = pd.concat([new_data_pandas_df, predictions_df], axis=1)

print("\n--- 입력 데이터와 최종 UHII 예측 결과 ---")
display(result_df)

In [0]:
# 예측 - 정적피쳐 확인

# --- 고정할 구와 시간/외곽 온도 설정 ---
target_district = 'Gangbuk-gu'
fixed_timestamp = '2025-06-01T16:00:00.000Z' # 여름 오후 4시
fixed_suburban_temp_current = 23.5 # 여름철 다소 시원한 외곽 온도

# --- 고정할 다른 변수들의 기준값 설정 (중구의 일반적인 특성 또는 임의의 기준) ---
# 여기서는 예시로 임의의 값을 사용하지만, 실제 데이터의 평균을 사용해도 좋습니다.
fixed_car_registration_count = 53324   # 중구 기준값
fixed_population_density = 13174.096 # 중구 기준값
fixed_avg_km_per_road_km = 14927.25044 # 중구 기준값

# --- 다양한 시나리오별 녹지율 및 건물 밀도 설정 ---
data_rows = []

# 시나리오 1: 중구의 현재(또는 기준) 특성
data_rows.append((
    target_district,
    0.255,   # 중구의 녹지율 기준
    2.07,    # 중구의 건물 밀도 기준
    fixed_car_registration_count,
    fixed_population_density,
    fixed_avg_km_per_road_km,
    fixed_timestamp,
    fixed_suburban_temp_current
))

# 시나리오 2: 녹지율을 낮게 (열섬 심화 예상)
data_rows.append((
    target_district,
    0.1,     # 녹지율 매우 낮게
    2.07,    # 건물 밀도 유지
    fixed_car_registration_count,
    fixed_population_density,
    fixed_avg_km_per_road_km,
    fixed_timestamp,
    fixed_suburban_temp_current
))

# 시나리오 3: 녹지율을 높게 (열섬 완화 예상)
data_rows.append((
    target_district,
    0.5,     # 녹지율 높게
    2.07,    # 건물 밀도 유지
    fixed_car_registration_count,
    fixed_population_density,
    fixed_avg_km_per_road_km,
    fixed_timestamp,
    fixed_suburban_temp_current
))

# 시나리오 4: 건물 밀도를 낮게 (열섬 완화 예상)
data_rows.append((
    target_district,
    0.255,   # 녹지율 유지
    1.0,     # 건물 밀도 낮게
    fixed_car_registration_count,
    fixed_population_density,
    fixed_avg_km_per_road_km,
    fixed_timestamp,
    fixed_suburban_temp_current
))

# 시나리오 5: 건물 밀도를 높게 (열섬 심화 예상)
data_rows.append((
    target_district,
    0.255,   # 녹지율 유지
    2.8,     # 건물 밀도 높게
    fixed_car_registration_count,
    fixed_population_density,
    fixed_avg_km_per_road_km,
    fixed_timestamp,
    fixed_suburban_temp_current
))

# 시나리오 6: 녹지율 높고, 건물 밀도 낮게 (열섬 최적 완화 시나리오)
data_rows.append((
    target_district,
    0.6,     # 녹지율 매우 높게
    0.8,     # 건물 밀도 매우 낮게
    fixed_car_registration_count,
    fixed_population_density,
    fixed_avg_km_per_road_km,
    fixed_timestamp,
    fixed_suburban_temp_current
))

# 시나리오 7: 녹지율 낮고, 건물 밀도 높게 (열섬 최악 심화 시나리오)
data_rows.append((
    target_district,
    0.05,    # 녹지율 매우 낮게
    3.0,     # 건물 밀도 매우 높게
    fixed_car_registration_count,
    fixed_population_density,
    fixed_avg_km_per_road_km,
    fixed_timestamp,
    fixed_suburban_temp_current
))


# --- 오직 new_data_spark_df_raw 변수에 위에서 생성한 data_rows를 할당합니다. ---
new_data_spark_df_raw = spark.createDataFrame(data_rows, schema=[
    'District', 'green_rate', 'Building_Density', 'car_registration_count',
    'population_density', 'avg_km_per_road_km', 'timestamp', 'suburban_temp_current'
])
# --- 변경 끝 ---

print("--- 단일 구(중구)의 고정 변수 변화에 따른 Raw Spark DataFrame ---")
new_data_spark_df_raw.show(truncate=False)
new_data_spark_df_raw.printSchema()

# --- 이후의 전처리 및 예측 파이프라인은 이전과 동일하게 연결됩니다 ---

# 2. 학습 데이터와 동일한 Spark DataFrame 전처리 적용
new_data_spark_df_processed = new_data_spark_df_raw.withColumn("green_rate", col("green_rate").cast("double")) \
                                                     .withColumn("Building_Density", col("Building_Density").cast("double")) \
                                                     .withColumn("car_registration_count", col("car_registration_count").cast("long")) \
                                                     .withColumn("population_density", col("population_density").cast("double")) \
                                                     .withColumn("avg_km_per_road_km", col("avg_km_per_road_km").cast("double")) \
                                                     .withColumn("timestamp", col("timestamp").cast("timestamp")) \
                                                     .withColumn("suburban_temp_current", col("suburban_temp_current").cast("double"))

print("\n--- 예측할 새로운 Spark DataFrame (전처리 후) ---")
new_data_spark_df_processed.show(truncate=False)
new_data_spark_df_processed.printSchema()


# 3. Spark DataFrame을 Pandas DataFrame으로 변환
new_data_pandas_df = new_data_spark_df_processed.toPandas()

print("\n--- 예측할 새로운 Pandas DataFrame ---")
display(new_data_pandas_df)
print("\nTimestamp 컬럼 타입 (Pandas):", new_data_pandas_df['timestamp'].dtype)


# 4. Pandas DataFrame을 H2OFrame으로 변환 (예측 데이터)
# best_model 객체는 이전에 학습이 완료되어 정의되어 있어야 합니다.
new_h2o_frame = h2o.H2OFrame(new_data_pandas_df)


# 5. H2OFrame의 'timestamp' 컬럼 타입 확인
h2o_frame_timestamp_type = new_h2o_frame.types.get('timestamp')
print(f"\n변환 후 예측 데이터 'timestamp' 컬럼 H2O 타입: {h2o_frame_timestamp_type}")

if h2o_frame_timestamp_type != 'time':
    print("경고: 'timestamp' 컬럼이 'time' 타입으로 인식되지 않았습니다. 모델 예측에 문제가 발생할 수 있습니다.")


print("\n--- 예측할 새로운 H2OFrame 정보 ---")
new_h2o_frame.show_summary()


# 6. 최고 성능 모델로 예측 수행
predictions = best_model.predict(new_h2o_frame)


print("\n--- 예측 결과 ---")
display(predictions.head(len(new_data_pandas_df))) # 모든 예측 결과 표시


# 7. 예측 결과를 Pandas DataFrame으로 변환하여 원본 데이터와 함께 보기
predictions_df = predictions.as_data_frame()
result_df = pd.concat([new_data_pandas_df, predictions_df], axis=1)


print("\n--- 입력 데이터와 최종 UHII 예측 결과 ---")
display(result_df)

In [0]:
# 예측 - 녹지율 변화에 따른 특정 구 예측

# 녹지율을 변화시킨 예측 데이터 생성
new_data_spark_df_raw_scenario2_green_rate = spark.createDataFrame([
    ('Jung-gu', 0.1,  jung_gu_avg_features['Building_Density'], jung_gu_avg_features['car_registration_count'], jung_gu_avg_features['population_density'], jung_gu_avg_features['avg_km_per_road_km'], TARGET_TIMESTAMP_STR, TARGET_SUBURBAN_TEMP), # 녹지율 매우 낮게 (예: 10%)
    ('Jung-gu', jung_gu_avg_features['green_rate'], jung_gu_avg_features['Building_Density'], jung_gu_avg_features['car_registration_count'], jung_gu_avg_features['population_density'], jung_gu_avg_features['avg_km_per_road_km'], TARGET_TIMESTAMP_STR, TARGET_SUBURBAN_TEMP), # 현재 중구 평균 녹지율
    ('Jung-gu', 0.4,  jung_gu_avg_features['Building_Density'], jung_gu_avg_features['car_registration_count'], jung_gu_avg_features['population_density'], jung_gu_avg_features['avg_km_per_road_km'], TARGET_TIMESTAMP_STR, TARGET_SUBURBAN_TEMP), # 녹지율 중간 수준으로 높게 (예: 40%)
    ('Jung-gu', 0.6,  jung_gu_avg_features['Building_Density'], jung_gu_avg_features['car_registration_count'], jung_gu_avg_features['population_density'], jung_gu_avg_features['avg_km_per_road_km'], TARGET_TIMESTAMP_STR, TARGET_SUBURBAN_TEMP)  # 녹지율 매우 높게 (예: 60%)
], schema=[
    'District', 'green_rate', 'Building_Density', 'car_registration_count',
    'population_density', 'avg_km_per_road_km', 'timestamp', 'suburban_temp_current'
])


In [0]:
# 모두 실행시 모델 등록 오류 발생 방지용 중단
raise Exception("여기서 중단합니다.")

## 새 모델 등록 (재실행 X)

In [0]:
# 모델 로깅
# 1. 모델 시그니처를 위한 샘플 입력 데이터 준비
# `input_example`은 Pandas DataFrame이어야 하며, H2O 모델의 학습 컬럼 스키마와 정확히 일치해야 합니다.
# 제공해주신 `new_data_spark_df_raw`의 컬럼들을 사용합니다.
sample_input = pd.DataFrame([
    ('Jungnang-gu', 0.364, 1.178, 113569, 21597.946, 8848.239521, '2020-08-01T18:00:00.000Z', 20.79),
    ('Dobong-gu', 0.569, 0.764, 96298, 15852.833, 5277.944143, '2020-08-01T18:00:00.000Z', 20.79),
    ('Jung-gu', 0.255, 2.07, 53324, 13174.096, 14927.25044, '2020-08-01T18:00:00.000Z', 20.79)
], columns=[
    'District', 'green_rate', 'Building_Density', 'car_registration_count',
    'population_density', 'avg_km_per_road_km', 'timestamp', 'suburban_temp_current'
])


# 2. H2O 모델의 예측 결과를 사용하여 출력 스키마 추론
# H2O 모델의 predict 메서드는 H2OFrame을 반환하므로, 이를 Pandas DataFrame으로 변환해야 합니다.
# 분류 모델의 경우 일반적으로 predict()는 확률(p0, p1 등)과 예측 레이블(predict)을 반환합니다.
# 실제 모델의 예측 결과 형태를 확인하여 `sample_output`을 구성해야 합니다.
try:
    # sample_input (Pandas DataFrame)을 H2OFrame으로 변환하여 predict 호출
    sample_output_hf = best_model.predict(h2o.H2OFrame(sample_input))
    sample_output = sample_output_hf.as_data_frame()
except Exception as e:
    print(f"Error predicting with H2O model for signature inference: {e}")
    print("Assuming a simple output for signature (e.g., single prediction column).")
    # 예외 발생 시, 최소한의 출력 스키마를 수동으로 가정
    # 실제 H2O 모델의 출력 형태에 맞게 수정해야 합니다.
    # 분류 모델이라면, 'predict' 컬럼과 'p0', 'p1' 등의 확률 컬럼이 있을 수 있습니다.
    # H2O 분류 모델의 기본 출력은 'predict' (클래스 레이블) 및 'p0', 'p1' (확률) 입니다.
    # 따라서 최소한 다음과 같이 구성해야 할 수도 있습니다.
    sample_output = pd.DataFrame({
        'predict': [0, 1],
        'p0': [0.5, 0.5],
        'p1': [0.5, 0.5]
    })


# 3. `infer_signature`를 사용하여 모델 시그니처 생성
# `infer_signature(model_input, model_output)`
signature = infer_signature(sample_input, sample_output)

# MLflow experiment 설정
mlflow.set_experiment("/Shared/H2O_AutoML_Deployment_Example")

# MLflow Run 시작
with mlflow.start_run(run_name="H2O_AutoML_Best_Model_Log_With_Signature") as run:
    run_id = run.info.run_id
    print(f"MLflow Run ID: {run_id}")

    mlflow.h2o.log_model(
        h2o_model=best_model,
        artifact_path="h2o_automl_best_model",
        conda_env={
            "channels": ["defaults", "conda-forge"],
            "dependencies": [
                "python=3.9", # 사용하고 있는 파이썬 버전으로 변경
                "pip",
                {
                    "pip": [
                        f"h2o=={h2o.__version__}", # 현재 설치된 H2O 버전 사용
                        "mlflow",
                        "scikit-learn", # H2O 모델이 scikit-learn 종속성을 가질 수 있음
                        "cloudpickle==2.0.0", # MLflow 모델 서빙을 위해 특정 버전 필요할 수 있음
                        "pandas" # 필요한 경우 추가
                    ]
                }
            ]
        },
        signature=signature, # <-- 이 부분이 수정된 핵심!
        input_example=sample_input # input_example은 여전히 유용하므로 유지
    )

    print(f"H2O best model logged to MLflow with signature at 'runs:/{run_id}/h2o_automl_best_model'")

mlflow.h2o.log_model: H2O 모델을 MLflow 형식으로 로깅하는 핵심 함수입니다.

h2o_model: 로깅할 H2O 모델 객체 (best_model).

artifact_path: MLflow UI에서 이 모델이 저장될 artifacts 경로입니다. (mlruns/<run_id>/artifacts/<artifact_path>)
conda_env: 매우 중요합니다. 이 conda_env는 MLflow 모델 엔드포인트가 모델을 로드하고 예측을 수행하는 데 필요한 Python 환경과 라이브러리 종속성을 정의합니다.

H2O 버전을 모델을 학습시킨 버전과 정확히 일치시켜야 합니다.


cloudpickle 버전도 특정 버전으로 고정하는 것이 모델 서빙에서 오류를 줄이는 데 도움이 됩니다.

모델 예측에 필요한 다른 라이브러리(예: pandas)가 있다면 추가합니다.

input_example: 모델의 입력 스키마를 정의하는 데 사용됩니다. 이는 모델 서빙 엔드포인트에서 요청 유효성 검사 및 문서화를 위해 유용합니다. 실제 모델의 입력 형태와 일치하는 Pandas DataFrame을 사용해야 합니다.

In [0]:
# 이전 셀의 `run_id`와 `artifact_path`를 사용하여 모델 등록
registered_model_name = "H2O_AutoML_predictor_Model" # 등록할 모델 이름

model_uri = f"runs:/{run_id}/h2o_automl_best_model"
registered_model = mlflow.register_model(
    model_uri=model_uri,
    name=registered_model_name
)

print(f"Model registered as: {registered_model.name} version {registered_model.version}")

## 기존 모델 재등록 (버전관리)

In [0]:
# 1. 새로운 Run에서 모델 시그니처를 위한 샘플 입력 데이터 준비
#    H2O 모델의 학습 컬럼 스키마와 정확히 일치해야 합니다.
sample_input = pd.DataFrame([
    ('Jungnang-gu', 0.364, 1.178, 113569, 21597.946, 8848.239521, '2020-08-01T18:00:00.000Z', 20.79)
], columns=[
    'District', 'green_rate', 'Building_Density', 'car_registration_count',
    'population_density', 'avg_km_per_road_km', 'timestamp', 'suburban_temp_current'
])

# 2. H2O 모델의 예측 결과를 사용하여 출력 스키마 추론
sample_output_hf = best_model.predict(h2o.H2OFrame(sample_input))
sample_output = sample_output_hf.as_data_frame()
signature = infer_signature(sample_input, sample_output)

# MLflow experiment 설정 (이전과 동일)
mlflow.set_experiment("/Shared/H2O_AutoML_Deployment_Example")

# 새로운 MLflow Run 시작
with mlflow.start_run(run_name="H2O_AutoML_New_Version_Log") as run:
    new_run_id = run.info.run_id # 새로운 Run ID
    print(f"새로운 MLflow Run ID: {new_run_id}")

    mlflow.h2o.log_model(
        h2o_model=best_model,
        artifact_path="h2o_automl_best_model",
        conda_env={
            "channels": ["defaults", "conda-forge"],
            "dependencies": [
                "python=3.9",
                "pip",
                {"pip": [f"h2o=={h2o.__version__}", "mlflow", "scikit-learn", "cloudpickle==2.0.0", "pandas"]}
            ]
        },
        signature=signature,
        input_example=sample_input
    )
    print(f"새 모델이 Runs에 로깅되었습니다: 'runs:/{new_run_id}/h2o_automl_best_model'")

In [0]:
# 기존에 등록된 모델 이름과 동일하게 지정
registered_model_name = "H2O_AutoML_predictor_Model"

# 새로운 모델의 MLflow URI (위에서 생성된 new_run_id 사용)
model_uri = f"runs:/{new_run_id}/h2o_automl_best_model"

# 동일한 이름으로 register_model 호출
# MLflow가 자동으로 다음 버전을 할당하여 등록합니다.
registered_model_version = mlflow.register_model(
    model_uri=model_uri,
    name=registered_model_name
)

print(f"모델 '{registered_model_name}'의 새로운 버전이 등록되었습니다: 버전 {registered_model_version.version}")

## 앙상블 (테스트) - 실패

In [0]:
# --- 정적 피처와 타겟 정의 ---
static_features = [
    'District', 'green_rate', 'Building_Density', 'car_registration_count',
    'population_density', 'avg_km_per_road_km'
]
target_feature = 'UHII'

# --- AutoML을 위한 H2OFrame 준비 (선택 사항: 명시적으로 static_features만 포함) ---
# 이렇게 하면 AutoML이 불필요한 컬럼을 학습하지 않습니다.
# 단, 원본 h2o_df에는 target_feature도 있어야 합니다.
static_h2o_training_frame = h2o_frame[:, static_features + [target_feature]]


# --- H2O AutoML 실행 (정적 피처만 사용하여) ---
print("\n--- 정적 피처만으로 H2O AutoML 학습 시작 ---")
# AutoML 객체 생성
# max_runtime_secs: AutoML이 학습할 최대 시간 (초 단위). 적절히 설정 (예: 600초 = 10분)
# seed: 재현성을 위한 시드
# sort_metric: 리더보드를 정렬할 기준 (회귀에서는 'rmse'가 적합)
static_automl_leaderboard = H2OAutoML(
    max_runtime_secs=600, # 10분 동안 학습
    seed=1234,
    sort_metric="rmse"
)

# 학습 시작
# x에는 정적 피처 리스트, y에는 타겟 피처 이름, training_frame에는 정적 피처와 타겟이 있는 H2OFrame
static_automl_leaderboard.train(
    x=static_features,
    y=target_feature,
    training_frame=static_h2o_training_frame
)

# 최적의 모델 (리더 모델) 가져오기
best_static_model_from_automl = static_automl_leaderboard.leader

print("\n--- H2O AutoML 정적 피처 모델 리더보드 ---")
static_automl_leaderboard.leaderboard.as_data_frame()


print(f"\n--- H2O AutoML로 찾은 최적의 정적 피처 모델: {best_static_model_from_automl.model_id} ---")
best_static_model_from_automl.show()
print(f"최적 Static Model RMSE (Cross-Validation / Leaderboard): {best_static_model_from_automl.rmse():.4f}")


In [0]:
predictions_static_model = best_static_model_from_automl.predict(new_h2o_frame).as_data_frame()
display(predictions_static_model)

In [0]:
# 예시 코드: 앙상블 예측
predictions_full_model = best_model.predict(new_h2o_frame).as_data_frame()
predictions_static_model = best_static_model_from_automl.predict(new_h2o_frame).as_data_frame()

# 두 예측 결과의 컬럼 이름을 통일 (예: predict)
predictions_full_model.columns = ['predict_full']
predictions_static_model.columns = ['predict_static']

# pandas DataFrame으로 결합
combined_predictions_df = pd.concat([predictions_full_model, predictions_static_model], axis=1)

# 가중 평균 (예: 80%는 풀 모델, 20%는 정적 모델의 영향)
weight_full = 0.6
weight_static = 0.4
combined_predictions_df['predict_ensemble'] = (
    weight_full * combined_predictions_df['predict_full'] +
    weight_static * combined_predictions_df['predict_static']
)

# 최종 결과 df에 결합하여 확인
result_df_ensemble = pd.concat([new_data_pandas_df, combined_predictions_df[['predict_ensemble']]], axis=1)
print("\n--- 앙상블 예측 결과 (정적 피처 모델 포함) ---")
display(result_df_ensemble)